# py-process 

In [ ]:
'''python实现多进程有2种方式:
1. os.fork()方法, 仅适用unix/linux系统, 不支持windows
2. multiprocessing模块, 跨平台
'''

'''说明: 调用fork函数返回2次, 因为操作系统要将父进程复制一个子进程, 这2个进程几乎完全相同, 
fork方法分别要在父进程和子进程中返回, 子进程永远返回0, 父进程返回子进程的pid
'''
# 使用fork方法创建进程
import os
print("current pid = {}".format(os.getpid()))
pid = os.fork()
if pid < 0:
    print("error in fork")
elif pid == 0:
    print("this is child process = {}, my parent process id = {}".format(
        os.getpid(), os.getppid()))
else:
    print("i {} created a child process {}".format(os.getpid(), pid))

# 使用multiprocess模块创建多进程
'''创建子进程只需要传入一个执行函数和函数的参数,就完成一个process实例的创建,
用start()方法启动进程,用join()方法实现进程间同步
如果在创建Process时不指定target，那么执行时没有任何效果。因为默认的run方法是判断如果不指定target，那就什么都不做
join方法的作用:阻塞当前进程，直到调用join方法的那个进程执行完，再继续执行当前进程。
使用多进程的常规方法是，先依次调用start启动进程，再依次调用join要求主进程等待子进程的结束。
 '''
import os
from multiprocessing import Process


def run_proc(name):
    print("child process {} {} is running...".format(name, os.getpid()))


if __name__ == "__main__":
    print("parent process is {}".format(os.getpid()))
    for i in range(5):
        p = Process(target=run_proc, args=(str(i),))
        print("process will start.")
        p.start()
    p.join()
    print("process end.")


# 进程池
# 如果要启动大量子进程,则使用进程池比较合适,默认进程池大小是cpu的核数
from multiprocessing import Pool
import os
import time
import random


def run_task(name):
    print("task {} (pid = {}) is running...".format(name, os.getpid()))
    time.sleep(random.random() * 3)
    print("task {} is end".format(name))


if __name__ == "__main__":
    print("current process pid = {}".format(os.getpid()))
    p = Pool(processes=3)
    for i in range(5):
        p.apply_async(run_task, args=(i, ))
    print("waiting for all subprocesses done...")
    p.close()
    # 注意: 调用join之前，先调用close函数，否则会出错。执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    p.join()
    print("all subprocess done")

# 函数解释：
# apply_async(func[, args[, kwds[, callback]]]) 它是非阻塞，
# apply(func[, args[, kwds]])是阻塞的
# close()    关闭pool，使其不在接受新的任务。
# terminate()    结束工作进程，不在处理未完成的任务。
# join()    主进程阻塞，等待子进程的退出， join方法要在close或terminate之后使用。


# 使用pool.map函数
import multiprocessing


def run_task(x):
    print(x * x)


if __name__ == '__main__':
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    i_list = range(8)
    pool.map(run_task, i_list)

# 进程间通信,主要使用Queue和Pipe两种方式
# Pipe主要用于2个进程间的通信
# Queue主要用于多个进程间的通信

'''Queue通信方式:
Put方法:将数据插入到队列中,该方法有2个可选参数:blocked和timeout.
blocked为true,且timeout为正值,则阻塞timeout指定的时间,知道队列有剩余空间,若超时则抛出Queue.Full异常.
blocked为false,若队列已满,则立刻抛出Queue.Full异常
Get方法:从队列读取并删除一个元素,该方法有2个可选参数:blocked和timeout.
blocked为true,且timeout为正值,若在timeout指定的等待时间内没有取到元素,则抛出Queue.Empty异常
blocked为false,若queue有一个值可用,则立即返回该值,若queue为空,则立即抛出Queue.Empty异常
'''

from multiprocessing import Process, Queue
import os, time, random

def proc_write(q, urls):
    print("process {} is writing...".format(os.getpid()))
    for url in urls:
        q.put(url)
        print("put {} to queue...".format(url))
        time.sleep(random.random())

def proc_read(q):
    print("process {} is reading...".format(os.getpid()))
    while True:
        url = q.get(True)
        print("get {} from queue".format(url))

if __name__ == "__main__":
    q = Queue()
    proc_write1 = Process(target=proc_write, args=(q, ['url_1', 'url_2', 'url_3']))
    proc_write2 = Process(target=proc_write, args=(q, ['url_4', 'url_5', 'url_6']))

    proc_reader = Process(target=proc_read, args=(q, ))

    proc_write1.start()
    proc_write2.start()

    proc_reader.start()

    proc_write1.join()
    proc_write2.join()
    # proc_reader进程是死循环,无法等待其结束,所以不能使用join, 只能强行终止
    proc_reader.terminate()
    

# pipe通信机制
# pipe常用来在两个进程间进行通信,分别位于管道的两端,Pipe方法返回(conn1, conn2),代表管道的两个端, Pipe方法有duplex参数,默认为true,即全双工模式
# duplex为true,则这个管道是全双工,即conn1和conn2均可收发.全双工模式下,调用conn1.send发送消息,conn1.recv接收消息,若没有消息可接收,recv方法会一直阻塞,
# 若管道被关闭,则conn1.recv方法会抛出EOFError
# duplex为false,则conn1只负责接收消息,conn2只负责发送消息
import multiprocessing
import os
import time
import random


def proc_send(pipe, urls):
    for url in urls:
        print("process {} send {}".format(os.getpid(), url))
        pipe.send(url)
        time.sleep(random.random())


def proc_recv(pipe):
    while True:
        print("process {} recev {}".format(os.getpid(), pipe.recv()))
        time.sleep(random.random())


if __name__ == "__main__":
    pipe = multiprocessing.Pipe()
    p1 = multiprocessing.Process(target=proc_send, args=(
        pipe[0], ['url_' + str(i) for i in range(10)]))
    p2 = multiprocessing.Process(target=proc_recv, args=(pipe[1], ))

    p1.start()

    p2.start()

    p1.join()
    # p2.join()
    p2.terminate()